### get raw data

In [ ]:
# Set workspace to Windrecorder dir
import os
os.chdir("..")
os.chdir("..")
# ------------------------------------------------------------

import datetime
import pandas as pd

from windrecorder.db_manager import db_manager
from windrecorder.oneday import OneDay
from windrecorder.record_wintitle import get_wintitle_stat_in_day

In [ ]:
dt_select = datetime.datetime.today()

In [ ]:
df_day_search_result = OneDay().search_day_data(
                    dt_select,
                    search_content="",
                )

df_day_search_result

In [ ]:
df_day_activity_raw, day_time_sum = get_wintitle_stat_in_day(dt_select, optimize_for_display=False)
df_day_activity_optimize_display, _ = get_wintitle_stat_in_day(dt_select, optimize_for_display=True)

df_day_activity_raw

### clean and format data

In [ ]:
df_day_search_result['videofile_time'] = pd.to_datetime(df_day_search_result['videofile_time'], unit='s').dt.strftime('%Y-%m-%d %H-%M-%S')

clean_df = pd.DataFrame(columns=['datetime', 'activity', 'ocr_content'])
activity_compare_lst = []
vaild_day_activity_lst = []
max_token_limit = 1000000/2.5
max_pre_row_token = int(max_token_limit/len(df_day_search_result))
activity_deduplication_trace_depth = 10
MIN_ACTIVITY_DURATION_SECOND = 30
MAX_DAY_ACTIVITY_HEAD_TRUNCATION_FOR_LLM = 20

for index, row in df_day_activity_raw.iterrows():
    if row['Screen Time'] < MIN_ACTIVITY_DURATION_SECOND:
        continue
    vaild_day_activity_lst.append(row['Page'])

for index, row in df_day_search_result.iterrows():
    activity = row['win_title']
    
    # 剔除过近的重复项
    if activity in activity_compare_lst[-activity_deduplication_trace_depth:] or activity not in vaild_day_activity_lst or len(activity.split(" | ")[0]) == 0:
        continue
    activity_compare_lst.append(activity)
    ocr_text = row['ocr_text']

    # 剔除时间过短的项
    pass
    
    row = {
        'datetime': row['videofile_time'],
        'activity': activity.split(" | ")[0],
        'ocr_content': row['ocr_text'][:max_pre_row_token],
    }
    clean_df.loc[len(clean_df )] = row

df_day_activity_optimize_display = df_day_activity_optimize_display.head(MAX_DAY_ACTIVITY_HEAD_TRUNCATION_FOR_LLM)

clean_df

### generate prompt context